# How it's calculated:
On the scale of 10, you'll be graded based on: 
- API credentials
- strings manipulation
- functions application
- dataframe manipulation

Try to use more functions and pandas built in functions. You can click tab to see what's inside of pandas: pd.[tab]

### Your final scores = 0.8 * [quiz1+quiz2+assign1 performance] + 0.2 * ec scores

I'll be accepting submissions until Nov 5, 2017.

# Problem: 
Use Google place api to gather price levels for restaurants. Get mean values for restaurants under the same zipcode.

1. Get price levels information through API:
```
yourkey = 
```

2. zip code is included in 'loc' column. 

```
example_loc = '4445 21ST ST, QUEENS, 11101'
example_zip = example_loc[-5:]
```

3. create a new 'zipcode' column inside of the dataframe. df is your pandas dataframe
```
df['zipcode'] = 
```

4. take price levels mean by zipcode. Ignore NaN
```
df.groupby('zipcode').mean()['google_price_level']
```


Google places api: https://developers.google.com/places/web-service/  

In [1]:
#__date__ = 'Oct 25, 2017'
import pandas as pd
import googlemaps
import json
import numpy as np
import requests

import warnings
warnings.filterwarnings("ignore")

In [17]:
# get inspected restaurant data
inspected = pd.read_csv('uniq_biz_yelp_geo_cut.csv', index_col=0)
inspected.head()

,CAMIS,DBA,loc,goog_lat,goog_lng
0,50063325,PARIS BAGUETTE,"575 8TH AVE, MANHATTAN, 10018",40.755052,-73.991831
1,50061140,TURNTABLE CHICKEN JAZZ,"20 W 33RD ST, MANHATTAN, 10001",40.748189,-73.986385
2,50060162,JANE,"515 ATLANTIC AVE, BROOKLYN, 11217",40.685265,-73.980549
3,50059345,WIN XIN RESTAURANT,"6001 14TH AVE, BROOKLYN, 11219",40.627298,-73.996322
4,50058634,LA FLOR DEL CANARIO #2 RESTAURANT PIZZERIA,"2225 PITKIN AVE, BROOKLYN, 11207",40.672729,-73.890212


In [8]:
# api credentials. Use your own 'secrets.json' file
with open('secrets.json') as f:    
    creds = json.load(f)
    google_client = googlemaps.Client(key=yourkey)

In [9]:
def get_keywords(name):
    keywords = name.split(' ')
    # lowercase
    keywords = map(lambda word: word.lower(), keywords)
    # ignore puncuation ect.
    keywords = map(lambda word: word.replace("'","")\
                                    .replace("-","")\
                                    .replace(".","")\
                                    .replace(",",""), keywords)
    # only keeps words of length 3 or more
    keywords = filter(lambda word: len(word) >= 3, keywords)
    # but throw out a common set of 3 letter words
    keywords = filter(lambda word: word not in ['and', 'the', 'for'], keywords)
    return keywords

In [21]:
# sample
get_keywords(inspected.loc[10,'DBA'])

['little', 'caesare']

In [11]:
def find_google(df):
    import tqdm
    for row, insp in tqdm.tqdm(df.iterrows()):
        df['google_categories'] = np.empty((len(df), 1))
        df['google_categories'][:] = np.nan
        df['google_categories'] = df['google_categories'].astype(object)
        
        name = insp[1]
        true_keywords = get_keywords(name)
        try:
            params = {
                'location': {'lat' : insp[3], 'lng' : insp[4]},
                'keyword': " ".join(true_keywords),
                'radius': 100,
            }

            google_results = google_client.places_nearby(**params)
            # finding the best match among candidates
            best_match = False
            for i, biz in enumerate(google_results['results']):
                candidate_keywords = get_keywords(biz['name'])
                union = len(set(true_keywords) & set(candidate_keywords))
                if union > 0:
                    if best_match == False or union > best_match[1]:
                        best_match = (i, union)
            # append restaurant attributes to the inspections dataframe
            if best_match == False:
                pass # dataframe attributes already default to 'None'
            else:
                best_match = google_results['results'][best_match[0]]
                df.loc[row, 'google_price_level'] = best_match.get('price_level', np.nan)
                types = [best_match['types']]
                df.set_value(row, 'google_categories', types )
                df.loc[row, 'google_rating'] = best_match.get('rating', np.nan)
                
        except (IndexError, requests.Timeout, googlemaps.exceptions.HTTPError):
            df.set_value(row, 'google_categories', None )
            df.set_value(row, 'google_price_level', None )
            df.set_value(row, 'google_rating', None )
    return df

In [24]:
# testing on the first five rows
wowgg = find_google(inspected.loc[:4,:])


6it [00:01,  3.39it/s]


In [25]:
wowgg

,CAMIS,DBA,loc,goog_lat,goog_lng,google_categories,google_price_level,google_rating
0,50063325,PARIS BAGUETTE,"575 8TH AVE, MANHATTAN, 10018",40.755052,-73.991831,NaN,NaN,4.8
1,50061140,TURNTABLE CHICKEN JAZZ,"20 W 33RD ST, MANHATTAN, 10001",40.748189,-73.986385,NaN,2.0,4.3
2,50060162,JANE,"515 ATLANTIC AVE, BROOKLYN, 11217",40.685265,-73.980549,NaN,NaN,NaN
3,50059345,WIN XIN RESTAURANT,"6001 14TH AVE, BROOKLYN, 11219",40.627298,-73.996322,NaN,1.0,4.1
4,50058634,LA FLOR DEL CANARIO #2 RESTAURANT PIZZERIA,"2225 PITKIN AVE, BROOKLYN, 11207",40.672729,-73.890212,NaN,NaN,3.0
5,50057842,DUNKIN DONUTS,"925 HUNTS POINT AVE, BRONX, 10459",40.820303,-73.891367,"[[bakery, cafe, restaurant, food, store, point...",1.0,3.9
